## scrape single pages from the Hungarian Film Archive
http://mandarchiv.hu/tart/jatekfilm

In [15]:
from bs4 import BeautifulSoup as soup, Tag
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random

import json

In [5]:
# 

USER_AGENTS = [
    # Chrome
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36',
    # Firefox
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:25.0) Gecko/20100101 Firefox/25.0',
    # Opera
    'Opera/9.80 (Windows NT 6.0) Presto/2.12.388 Version/12.14',
    # Safari
    'Mozilla/5.0 (iPad; CPU OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/6.0 Mobile/10A5355d Safari/8536.25',
    # Internet Explorer
    'Mozilla/5.0 (compatible; MSIE 10.6; Windows NT 6.1; Trident/5.0; InfoPath.2; SLCC1; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729; .NET CLR 2.0.50727) 3gpp-gba UNTRUSTED/1.0',
]

def get_header(agents):
    return {'User-agent': random.choice(agents)}

In [8]:
url = 'http://mandarchiv.hu/tart/jatekfilm?name=jatekfilm&action=film&id=255012030'
response = requests.get(url, headers=get_header(USER_AGENTS))
soup = BeautifulSoup(response.content, "html.parser")

In [9]:
# necessary lists
title = []
year = []
director = []
cinematographer = []
writer = []
editor = []
producer = []



# title
input_title = soup.find("div", {"class": "w_480"}).findNext('h2')
for T in input_title:
    T_temp = input_title.text
    title.append(T_temp)
    
# year
input_year = soup.find('p', {'class': 'date'})
for Y in input_year:
    Y_temp = input_year.text
    Y_temp = Y_temp[-5:]
    year.append(Y_temp)
    
# director
for d in soup.findAll(text='rendező / director:'):
    for item in d.parent.next_siblings:
        if isinstance(item, Tag):
            if 'class' in item.attrs and 'name' in item.attrs['class']:
                break
            d_body = item.text
            director.append(d_body)
            
# writer
for w in soup.findAll(text='forgatókönyvíró / writer (screenplay):'):
    for item in w.parent.next_siblings:
        if isinstance(item, Tag):
            if 'class' in item.attrs and 'name' in item.attrs['class']:
                break
            w_body = item.text
            writer.append(w_body)
    
# cinematographer
for c in soup.findAll(text='operatőr / cinematographer:'):
    for item in c.parent.next_siblings:
        if isinstance(item, Tag):
            if 'class' in item.attrs and 'name' in item.attrs['class']:
                break
            c_body = item.text
            cinematographer.append(c_body)
    
# editor
for e in soup.findAll(text='vágó / editor:'):
    for item in e.parent.next_siblings:
        if isinstance(item, Tag):
            if 'class' in item.attrs and 'name' in item.attrs['class']:
                break
            e_body = item.text
            editor.append(e_body)
    
# producer
for t in soup.findAll(text='producer / producer:'):
    for item in t.parent.next_siblings:
        if isinstance(item, Tag):
            if 'class' in item.attrs and 'name' in item.attrs['class']:
                break
            p_body = item.text
            producer.append(p_body)

In [30]:
# construct json for export
data = {}

data['film_id_1001'] = []
data['film_id_1001'].append({
    'title' : title,
    'year' : year,
    'director' : director,
    'writer' : writer,
    'cinematographer' : cinematographer,
    'editor' : editor,
    'producer' : producer
})


with open('../data/film_id_1001.txt', 'w') as outfile:
    json.dump(data, outfile)

In [31]:
data

{'film_id_1001': [{'title': ['Glass Tiger'],
   'year': [' 2001'],
   'director': ['Kapitány, Iván', 'Rudolf, Péter'],
   'writer': ['Hársing, Hilda',
    'Búss Gábor Olivér',
    'Kapitány, Iván',
    'Rudolf, Péter'],
   'cinematographer': ['Kapitány, Iván'],
   'editor': ['Vass Csaba'],
   'producer': ['Kovács, Gábor', 'Pataki, Ági']}]}

In [14]:
movie_dict

{'title': ['Glass Tiger'],
 'year': [' 2001'],
 'director': ['Kapitány, Iván', 'Rudolf, Péter'],
 'cinematographer': ['Kapitány, Iván'],
 'editor': ['Vass Csaba'],
 'producer': ['Kovács, Gábor', 'Pataki, Ági'],
 'writer': ['Hársing, Hilda',
  'Búss Gábor Olivér',
  'Kapitány, Iván',
  'Rudolf, Péter']}

In [12]:
raw = {'year' : pd.Series(year),\
          'title' : pd.Series(title),\
          'director' : pd.Series(director),\
          'writer' : pd.Series(writer),\
          'cinematographer' : pd.Series(cinematographer),\
          'editor' : pd.Series(editor),\
          'producer' : pd.Series(producer)
       }
raw_df = pd.DataFrame(raw)
raw_df

,year,title,director,writer,cinematographer,editor,producer
0,2001,Glass Tiger,"Kapitány, Iván","Hársing, Hilda","Kapitány, Iván",Vass Csaba,"Kovács, Gábor"
1,NaN,NaN,"Rudolf, Péter",Búss Gábor Olivér,NaN,NaN,"Pataki, Ági"
2,NaN,NaN,NaN,"Kapitány, Iván",NaN,NaN,NaN
3,NaN,NaN,NaN,"Rudolf, Péter",NaN,NaN,NaN
